In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import sys

In [44]:
def scrap_india_today():
    df_features = pd.DataFrame(columns = ["title", "link", "Content"])
    url = "https://www.indiatoday.in/news.html"
    coverpage_request = requests.get(url)
    if coverpage_request.status_code != 200:
         sys.exit("Coverpage request couldn't be finished")
    coverpage = coverpage_request.content
    coverpage_soup = BeautifulSoup(coverpage, "html5lib")
    #Extracting the news section of the page
    coverpage_news = coverpage_soup.find_all("div", attrs= {"data-id" : "news-section"})
    #Iterating through each news section
    session = requests.session()
    all_titles = []
    all_links = []
    all_content = []
    i=0
    for news_section in coverpage_news:


        #The structure of the page is such that the first headline has <h3> tag and following few headlines have <p> tag

        #First extracting the <h3> tag
        title = ""
        link = ""
        final_article = ""
        title = news_section.find("h3")["title"]
        link = "https://www.indiatoday.in" + news_section.find("h3").find("a")["href"]
        if ("/video/" not in link) and ("/photo/" not in link):
            #Introducing delay to be polite with the server
            t0 = time.time()
            final_article = link_to_content(link, session)
            response_delay = time.time() - t0
            time.sleep(5*response_delay)
            #Pushing to dataframe
            df_features = push_to_df(df_features, title=title, link=link, content=final_article)

        #Next extracting the <p> tags
        headlines = news_section.find_all("p")
        for headline in headlines:
            title = ""
            link = ""
            final_article = ""
            title = headline["title"]
            link = "https://www.indiatoday.in" + headline.find("a")["href"]
            if ("/video/" not in link) and ("/photo/" not in link):
                
                #Introducing delay to be polite with the server
                t0 = time.time()
                final_article = link_to_content(link, session)
                response_delay = time.time() - t0
                time.sleep(5*response_delay)
                #Pushing to dataframe
                df_features = push_to_df(df_features, title=title, link=link, content=final_article)
    return df_features

In [6]:
def link_to_content(link, session):
    article_page = session.get(link)
    article_content = article_page.content
    soup_article = BeautifulSoup(article_content, "html5lib")
    articleBody = soup_article.find("div", attrs = {"itemprop" : "articleBody"})
    #searching for html with structure <p> text </p> only
    list_paras = articleBody.find_all("p")
    final_article = ""
    for p in list_paras:
        #To remove all ads with structure like <p><a>.........</a><p>
        if len(p.find_all())!=0:
            continue
        final_article += p.get_text()
    return final_article

In [43]:
def push_to_df(df, title, link, content):
    df2 = pd.DataFrame({"title": [title], "link" : [link], "Content" : [content]})
    df = df.append(df2, ignore_index = True)
    return df

In [47]:
df_features = scrap_india_today()

In [48]:
df_features

,title,link,Content
0,How to prepare for new life after coronavirus ...,https://www.indiatoday.in/coronavirus-outbreak...,The world has survived all pandemics of the pa...
1,Manipur's Konsakhul sets example by distributi...,https://www.indiatoday.in/india/story/manipur-...,Konsakhul - a small village located in Manipur...
2,"‘Like how everyone’s muted,’ advocate, judge s...",https://www.indiatoday.in/india/story/bombay-h...,Justice Gautam Patel of the Bombay High Court ...
3,"Tamil Nadu equipped with enough PPEs, order pl...",https://www.indiatoday.in/india/story/tamil-na...,Tamil Nadu Chief Minister Edappadi K Palanisam...
4,Coronavirus: GB Pant Hospital removed from lis...,https://www.indiatoday.in/india/story/coronavi...,"GB Pant Hospital, a designated Covid-19 facili..."
5,SC order for free coronavirus testing may hind...,https://www.indiatoday.in/india/story/sc-order...,A decision by the Supreme Court of India to ma...
6,"Abused, thrashed by cops: AIIMS-Bhopal doctors",https://www.indiatoday.in/india/story/abused-a...,The Resident Doctor's Association has alleged ...
7,"Coronavirus: Priyanka pledges $100,000 to four...",https://www.indiatoday.in/movies/celebrities/s...,Actor Priyanka Chopra on Thursday pledged $100...
8,Farhan on gaining 15 kilos in six weeks for To...,https://www.indiatoday.in/movies/bollywood/sto...,Farhan Akhtar who stunned his fans with his ch...
9,Dhanush's sister misses family reunion during ...,https://www.indiatoday.in/movies/regional-cine...,Dhanush and his family have the habit of condu...
